In [179]:

from langgraph.graph import StateGraph, END
from typing import TypedDict
import pandas as pd
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
import os
import re


In [180]:
class AgentState(TypedDict):
    goal: str
    plan: str
    data_summary: str
    insights: str
    decisions: str
    report: str


In [181]:

load_dotenv()

True

In [184]:
llm = GoogleGenerativeAI(model="models/gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"])


In [ ]:
def planner_agent(state: AgentState):
    prompt = f"""
    You are a planning agent.
    Goal: {state['goal']}

    Create a step-by-step plan to analyze sales data from a CSV file
    and generate business recommendations.
    """
    plan = llm.invoke(prompt)  # returns string directly
    return {"plan": plan}


In [ ]:
def data_analysis_agent(state: AgentState):
    # Safe CSV read
    try:
        df = pd.read_csv(r"C:\Users\Admin\Desktop\Superstore.csv", encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(r"C:\Users\Admin\Desktop\Superstore.csv", encoding="latin1")

    summary = f"""
    Rows: {len(df)}
    Columns: {list(df.columns)}
    Total Sales: {df['Sales'].sum()}
    Average Sales: {df['Sales'].mean():.2f}
    Top Region: {df.groupby('Region')['Sales'].sum().idxmax()}
    Top Product: {df.groupby('Product Name')['Sales'].sum().idxmax()}
    """

    return {"data_summary": summary}


In [ ]:
def insight_agent(state: AgentState):
    prompt = f"""
    You are a senior data analyst AI.

    Based on the following data summary:
    {state['data_summary']}

    Identify key insights, trends, and anomalies.
    """
    insights = llm.invoke(prompt)  # string directly
    return {"insights": insights}


In [ ]:
def decision_agent(state: AgentState):
    prompt = f"""
    You are a business strategy AI.

    Insights:
    {state['insights']}

    Suggest clear, actionable steps to improve revenue and efficiency.
    """
    decisions = llm.invoke(prompt)  # returns string directly
    return {"decisions": decisions}


In [ ]:
def report_agent(state: AgentState):
    report = f"""
    📊 AUTONOMOUS SALES ANALYSIS REPORT

    🎯 Goal:
    {state['goal']}

    🧠 Plan:
    {state['plan']}

    📈 Data Summary:
    {state['data_summary']}

    🔍 Insights:
    {state['insights']}

    🚀 Business Decisions:
    {state['decisions']}
    """
    return {"report": report}


In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("planner", planner_agent)
workflow.add_node("analysis", data_analysis_agent)
workflow.add_node("insights", insight_agent)
workflow.add_node("decisions", decision_agent)
workflow.add_node("report", report_agent)

workflow.set_entry_point("planner")

workflow.add_edge("planner", "analysis")
workflow.add_edge("analysis", "insights")
workflow.add_edge("insights", "decisions")
workflow.add_edge("decisions", "report")
workflow.add_edge("report", END)

agent_app = workflow.compile()


In [ ]:
result = agent_app.invoke({
    "goal": "Analyze sales CSV data and recommend revenue growth strategies"
})

print(result["report"])